In [1]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device='mps')

hypothesis_ne = "The statement '{}' downplays, denies, or avoids"
hypothesis_nne = "The statement '{}' acknowledges"

# Labels
targets = ["Yes", "No"]

#EZ-STANCE: A Large Dataset for English Zero-Shot Stance Detection

Device set to use mps


In [2]:
import pandas as pd
import ast
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import numpy as np
import math
future = pd.read_csv("./data/ca_futre_labels.csv", header=None).iloc[:,0].to_list()

idxx = pd.read_csv("./data/ca_future.csv", encoding="latin1").dropna(how='all').sentence_id.astype(int)
# for model:predictions
labels = pd.read_csv("./data/ca_future.csv", encoding="latin1").dropna(how='all').targets
sentences = pd.read_csv("./data/ca_future.csv", encoding="latin1").dropna(how='all').sentence_text
labels = [ast.literal_eval(str(i)) if not isinstance(i, float) or not math.isnan(i) else [0 for ii in range(11)] for i in labels]

In [3]:
statement_idx = pd.read_csv("./data/ca_future.csv", encoding="latin1").dropna(how='all').statement_id.tolist()

In [4]:
not_used = pd.read_csv("./data/ca_future.csv" , encoding="latin1").dropna(how='all')
not_used_id = not_used.iloc[:,7].apply(lambda x: "negative" in str(x))
negative_instances = not_used[not_used_id].sentence_id.astype(int).tolist()

In [5]:
import math
task_future_labels = []
st = []
for sentence, data, i in zip(sentences, np.array(labels), idxx):
    
    for ii, lbs in enumerate(data[3:]):
        if lbs == 1:
            if i in negative_instances:
                task_future_labels.append(1)
            else:
                task_future_labels.append(0)
            st.append(sentence)

In [6]:
len(negative_instances)

101

In [7]:
len(labels)

3658

In [8]:
101/len(labels)

0.02761071623838163

In [9]:
labels_name = [
    "approval",
    "signature",
    "c1 (reporting entity)",
    "c2 (structure)",
    "c2 (operations)",
    "c2 (supply chains)",
    "c3 (risk description)",
    "c4 (risk mitigation)",
    "c4 (remediation)",
    "c5 (effectiveness)",
    "c6 (consultation)"
]

In [10]:
import numpy as np
import tqdm

results = {key:[] for key in labels_name[3:]}
predicted_results = []
neg_scores = []
pos_scores = []
ct = {labels_name[idx+2]:0 for idx in range(9)}
st_per_l = {labels_name[idx+2]:[] for idx in range(9)}
score = []
st_list = []
new_prediction_list = []
sen_id_list = []
stat_id_list = []
for sentence, data, i, si in zip(sentences, np.array(labels),idxx,statement_idx):
    for idx, lbs in enumerate(data[3:]):
        if lbs == 1:
            result_ne = classifier(sentence, targets, hypothesis_template=hypothesis_ne)
            result_nne = classifier(sentence, targets, hypothesis_template=hypothesis_nne)
  
            pos = result_ne['scores'][0] if result_ne['labels'][0] == targets[0] else result_ne['scores'][1]
            neg = result_nne['scores'][0] if result_nne['labels'][0] == targets[0] else result_nne['scores'][1]
            pos_scores.append(pos)
            neg_scores.append(neg)
            st_list.append(sentence)
            new_prediction_list.append(data)
            if pos - neg >0.15:
                predicted_results.append(1)
                ct[labels_name[idx+2]] += 1
                st_per_l[labels_name[idx+2]].append(sentence)
                score.append(pos/(pos+neg))
            else:
                predicted_results.append(0)
                score.append(neg/(pos+neg))
            sen_id_list.append(i)
            stat_id_list.append(si)

In [11]:
len(st_list)

2179

In [12]:
import numpy as np
from sklearn.metrics import f1_score

print(f1_score(task_future_labels, predicted_results, average='micro')) 
print(f1_score(task_future_labels, predicted_results, average='weighted'))
print(f1_score(task_future_labels, predicted_results, average='macro'))

0.9366681964203764
0.9353436860974167
0.6779913303463578
